<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/LLM_Court.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import torch
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import time
import re
import openai
import json
from sklearn.model_selection import train_test_split
import ast

In [2]:

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

folder_id = "בית המשפט - קבצים שנבדקו"
directory_path = f"/content/drive/My Drive/{folder_id}"
classifiers_path = "/content/drive/MyDrive/בית המשפט - מסווגים/"

if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the folder path.")
else:
    dataframes = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)
            try:
                df = pd.read_excel(file_path)
                dataframes.append(df)
            except Exception as e:
                print(f"Failed to read {file_name}: {e}")

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("All Excel files have been concatenated successfully!")

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


All Excel files have been concatenated successfully!


In [4]:
len(dataframes)

6

In [5]:
df_1 = dataframes[0]

df_1.columns

Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
       'prediction_original_model', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'קידוד', 'קידוד חוזר', 'הערות ',
       'הערות קידוד חוזר'],
      dtype='object')

In [6]:
df_1 = df_1[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',  'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ',
       'הערות קידוד חוזר']]

In [7]:
df_1 = df_1[~(df_1["הערות "].isna() & df_1["הערות קידוד חוזר"].isna() & df_1["טקסט שמצביע על תוצאת בקשה שונה"].isna())]

In [8]:
df_2 = dataframes[1]
df_2.columns

Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
       'prediction_custom_threshold', 'prediction_original_model',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'רע"פ בקשה אחרת או דלמטה מיוחד', 'קידוד', 'קידוד חוזר',
       'הערות תוצאת ערעור', 'הערות ', 'הערות קידוד חוזר'],
      dtype='object')

In [9]:
df_2 = df_2[['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'רע"פ בקשה אחרת או דלמטה מיוחד',
       'הערות תוצאת ערעור', 'הערות ', 'הערות קידוד חוזר']]

In [10]:
df_2 = df_2[~(df_2["הערות "].isna() & df_2['הערות תוצאת ערעור'].isna() & df_2["הערות קידוד חוזר"].isna() & df_2['טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)'].isna())]

In [11]:
df = pd.concat([df_1, df_2], ignore_index=True)

In [12]:
df_3 = dataframes[2]
df_3.columns

Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
       'prediction_original_model',
       'prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?',
       'גוף המסמך חתוך מסווג שני', 'prediction_second_model',
       'second_model_predicted_probability',
       'prediction_second_model_adjusted\nערעור התקבל?',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'הערות תוצאת ערעור', 'רע"פ בקשה אחרת או דלמטה מיוחד',
       'קידוד', 'הערות '],
      dtype='object')

In [13]:
df_3 = df_3[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',
       'prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?',
       'גוף המסמך חתוך מסווג שני',
       'prediction_second_model_adjusted\nערעור התקבל?',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'הערות תוצאת ערעור', 'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ']]

In [14]:
df_3 = df_3[~(df_3["הערות "].isna() & df_3['הערות תוצאת ערעור'].isna() & df_3['טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)'].isna())]

In [15]:
df = pd.concat([df, df_3], ignore_index=True)

In [16]:
df[~df["טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)"].isna()]

,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,בקשה לרשות ערעור התקבלה?,טקסט שמצביע על תוצאת בקשה שונה,הערעור התקבל?,"רע""פ בקשה אחרת או דלמטה מיוחד",הערות,הערות קידוד חוזר,level_0,index,טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**),הערות תוצאת ערעור,prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?,גוף המסמך חתוך מסווג שני,prediction_second_model_adjusted\nערעור התקבל?
89,"['פסק-דין בתיק רע""פ 5800/04 בבית המשפט העליון ...","רע""פ 5800/04","רע""פ 5800/04 מחמד סוויטי נ. התובע הצבאי",2004,"ד השופטים: אל""ם ש\' גורדון, אל""ם ד\' פרידמן, ס...",אחר,NaN,NaN,דלמטה מיוחד,קשת רשות ערעור על פסק דינו של בית המשפט הצבאי ...,NaN,14.0,5440.0,אור תשובתהמשיב ביקש בא כוח המבקש למחוק הבקשה. ...,NaN,NaN,NaN,NaN
95,"['פסק-דין בתיק רע""פ 10143/04 בבית המשפט העליון...","רע""פ 10143/04","רע""פ 10143/04 קליינר הנדסה בע""מ נ. מינהל מקרקע...",2004,"החלטת הקנס, ולאור עמדתו של המשיב, מצאנו כי יש ...",חלקי,NaN,חלקי,NaN,NaN,NaN,100.0,5533.0,בשאלה זו האחרונה החלטנו להתערב. ועל-כן אנו מקב...,"יחד עם זאת, לאור גובהו של הקנס שהוטל עלהמבקשת,...",NaN,NaN,NaN
97,"['פסק-דין בתיק רע""פ 9479/04 בבית המשפט העליון ...","רע""פ 9479/04","רע""פ 9479/04 יוספ דידו נ. מדינת ישראל",2004,"ת. בשים לב לגילוולמצב בריאותו של המבקש, באו בא...",חלקי,NaN,חלקי,NaN,NaN,נראה שרשות הערעור התקבלה והערעור התקבל חלקית,145.0,5582.0,"עונש המאסר על תנאי, על-פי אותוהסדר, נותר בעינו",NaN,NaN,NaN,NaN
98,"['פסק-דין בתיק רע""פ 6198/04 בבית המשפט העליון ...","רע""פ 6198/04","רע""פ 6198/04 רונ דורונ נ. מדינת ישראל",2004,ורון נ ג ד המשיבה: מדינת ישראל בקשת רשות ערעור...,אחר,NaN,NaN,NaN,הבקשה נמחקה בהצעת בית המשפט,NaN,153.0,5592.0,"בהמלצתבית המשפט, חזר בו המבקש מבקשתו והיא נמחקת.",NaN,NaN,NaN,NaN
99,"['פסק-דין בתיק רע""פ 2491/04 בבית המשפט העליון ...","רע""פ 2491/04","רע""פ 2491/04 יורמ דינור נ. מדינת ישראל",2004,ד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק ד...,אחר,NaN,NaN,NaN,הבקשה נמחקה,NaN,188.0,5631.0,"בהמלצתנו, חזרבו המבקש מהבקשה ואנו מוחקים אותה.",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,"['החלטה בתיק רע""פ 1995/17 בבית המשפט העליון רע...","רע""פ 1995/17","רע""פ 1995/17 יונס עבד אל לטיפ נאדר נ. מדינת יש...",2017,כי מתן ההיתר לחלקו הרלבנטי של הנכס אינו מצוי ב...,נדחה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,בקשת רשות הערעור נדחית בזאת,NaN,1.0,"להצביע על ""אופק תכנוני"" מוצק די הצורך, ככל שהד...",0.0
322,"['החלטה בתיק רע""פ 4137/14 בבית המשפט העליון רע...","רע""פ 4137/14","רע""פ 4137/14",2014,ונית בקשת רשות ערעור על פסק דינו של בית המשפט ...,אחר,NaN,NaN,NaN,NaN,NaN,NaN,NaN,נדחה לעיון ולהחלטה,NaN,0.0,NaN,NaN
323,"['פסק דין בתיק רע""פ 4131/14 בבית המשפט העליון ...","רע""פ 4131/14","רע""פ 4131/14 יוספ נג'אר נ. מדינת ישראל",2014,") 747, 762-760 (1999). המבקש קיבל את המלצנו. ה...",אחר,NaN,NaN,NaN,NaN,NaN,NaN,NaN,המבקש קיבל את המלצנו. הבקשה נמחקת איפוא.,NaN,0.0,NaN,NaN
329,"['פסק דין בתיק רע""פ 4753/08 בבית המשפט העליון ...","רע""פ 4753/08","רע""פ 4753/08 דמיטרי סנקובסקי נ. מדינת ישראל",2008,אנו מחליטים כדלקמן: בקשת רשות הערעור של המבקש...,התקבל,NaN,התקבל,NaN,NaN,NaN,NaN,NaN,"בקשת רשות הערעור של המבקש מתקבלת,","בקשת רשות הערעור של המבקש מתקבלת, וההחלטה המשל...",0.0,NaN,NaN


In [17]:
df.shape

(348, 18)

In [18]:
len(df["גוף המסמך"])

# בדיקת האורך של כל פסק דין (במילים)
df["length"] = df["גוף המסמך"].apply(lambda x: len(str(x).split()))

# חישוב האורך הממוצע
average_length = df["length"].mean()
max_length = df["length"].max()
min_length = df["length"].min()

print(f"🔹 אורך ממוצע של פסקי דין: {average_length:.2f} מילים")
print(f"🔹 פסק הדין הארוך ביותר מכיל {max_length} מילים")
print(f"🔹 פסק הדין הקצר ביותר מכיל {min_length} מילים")


🔹 אורך ממוצע של פסקי דין: 722.41 מילים
🔹 פסק הדין הארוך ביותר מכיל 5920 מילים
🔹 פסק הדין הקצר ביותר מכיל 60 מילים


In [19]:


def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            text_list = ast.literal_eval(text)
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    elif isinstance(text, str):
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

print("שורות לא קריאות לפני המרה:")
print(df["גוף המסמך"].head())

df["גוף המסמך"] = df["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(df["גוף המסמך"].head())

file_path_full = 'checked_df_cleaned.csv' # FULL RAP DATA CLEANED
df.to_csv(file_path_full, index=False, encoding='utf-8')


שורות לא קריאות לפני המרה:
0    ['החלטה בתיק רע"פ 3622/15 בבית המשפט העליון רע...
1    ['החלטה בתיק רע"פ 825/15 בבית המשפט העליון רע"...
2    ['החלטה בתיק רע"פ 54/15 בבית המשפט העליון רע"פ...
3    ['פסק דין בתיק רע"פ 2811/15 בבית המשפט העליון ...
4    ['פסק דין בתיק רע"פ 2195/15 בבית המשפט העליון ...
Name: גוף המסמך, dtype: object

שורות לאחר המרה:
0    ['החלטה בתיק רע"פ 3622/15 בבית המשפט העליון רע...
1    ['החלטה בתיק רע"פ 825/15 בבית המשפט העליון רע"...
2    ['החלטה בתיק רע"פ 54/15 בבית המשפט העליון רע"פ...
3    ['פסק דין בתיק רע"פ 2811/15 בבית המשפט העליון ...
4    ['פסק דין בתיק רע"פ 2195/15 בבית המשפט העליון ...
Name: גוף המסמך, dtype: object


In [20]:
def extract_relevant_part(text):
    words = str(text).split()
    if len(words) > 1000:
        return " ".join(words[-1000:-20])
    return text

df["גוף המסמך חתוך"] = df["גוף המסמך"].apply(extract_relevant_part)

df[["גוף המסמך", "גוף המסמך חתוך"]].head()


,גוף המסמך,גוף המסמך חתוך
0,"['החלטה בתיק רע""פ 3622/15 בבית המשפט העליון רע...","בעניינו של המבקש, וזאת בשל אי-הבעת נזקקות מציד..."
1,"['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""...","['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""..."
2,"['החלטה בתיק רע""פ 54/15 בבית המשפט העליון רע""פ...",הוגשו שלושה כתבי אישום: כתב אישום נגד 7 בגירים...
3,"['פסק דין בתיק רע""פ 2811/15 בבית המשפט העליון ...","['פסק דין בתיק רע""פ 2811/15 בבית המשפט העליון ..."
4,"['פסק דין בתיק רע""פ 2195/15 בבית המשפט העליון ...","['פסק דין בתיק רע""פ 2195/15 בבית המשפט העליון ..."


In [21]:
df = df[['גוף המסמך', 'מספר הליך', 'שם הליך', 'שנת פתיחת ההליך',
       'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ', 'הערות קידוד חוזר',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערות תוצאת ערעור', 'length',]]

In [22]:
df.columns = df.columns.str.strip()  # מסיר רווחים מההתחלה והסוף
df.columns = df.columns.str.replace("\n", "")  # מסיר מעברי שורה
df.columns = df.columns.str.replace("  ", " ")  # מסיר רווחים כפולים

In [23]:
df.rename(columns={
    'הערות ':"הערות",
    'טקסט שמצביע על תוצאת בקשה שונה':"טקסט שמצביע על תוצאת בקשה",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)": "טקסט שמצביע על התוצאה"
}, inplace=True)

df.columns


<ipython-input-23-e10c7136f4af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={


Index(['גוף המסמך', 'מספר הליך', 'שם הליך', 'שנת פתיחת ההליך',
       'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות', 'הערות קידוד חוזר',
       'טקסט שמצביע על התוצאה', 'הערות תוצאת ערעור', 'length'],
      dtype='object')

In [24]:

columns_to_merge = [
    'טקסט שמצביע על תוצאת בקשה',
    'רע"פ בקשה אחרת או דלמטה מיוחד',  "הערות", 'הערות קידוד חוזר',
    "טקסט שמצביע על התוצאה",
    'הערות תוצאת ערעור',  'בקשה לרשות ערעור התקבלה?',"הערעור התקבל?"
]


# פונקציה לשילוב נתונים תוך שמירה על קריאות וניקוי כפילויות
def clean_merge_columns(row, columns):
    values = []
    for col in columns:
        if pd.notna(row[col]) and str(row[col]).strip():  # מתעלמים מערכים ריקים
            values.append(f"{col}: {str(row[col]).strip()}")  # מוסיפים תווית לכל עמודה

    return " || ".join(values) if values else None  # מפרידים בין הנתונים עם ||

# יצירת עמודה מאוחדת עם כותרות לכל ערך
df["תוצאה מסכמת"] = df.apply(lambda row: clean_merge_columns(row, columns_to_merge), axis=1)


def clean_text_result(text):
    if pd.isna(text):  # אם הערך ריק, נחזיר אותו כפי שהוא
        return text

    # החלפת סימני שאלה בשפה ברורה
    text = text.replace("?", "")

    # מחיקת "הערה של ___" אבל השארת ההמשך של המשפט
    text = re.sub(r"הערה של \S+\s*", "", text)

    return text.strip()  # מסירים רווחים מיותרים

# החלת הפונקציה על העמודה "תוצאה מסכמת"
df["תוצאה מסכמת"] = df["תוצאה מסכמת"].apply(clean_text_result)

# הצגת מספר דוגמאות לבדיקה
df[["תוצאה מסכמת"]].head()


<ipython-input-24-f520bf0deb0a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["תוצאה מסכמת"] = df.apply(lambda row: clean_merge_columns(row, columns_to_merge), axis=1)
<ipython-input-24-f520bf0deb0a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["תוצאה מסכמת"] = df["תוצאה מסכמת"].apply(clean_text_result)


,תוצאה מסכמת
0,טקסט שמצביע על תוצאת בקשה: לנוכח כל האמור לעיל...
1,טקסט שמצביע על תוצאת בקשה: בקשה לעיכוב ביצוע ע...
2,"טקסט שמצביע על תוצאת בקשה: אשר על כן, הבקשה לר..."
3,הערות: - זהו פסק דין משלים || בקשה לרשות ערעור...
4,טקסט שמצביע על תוצאת בקשה: חזרו בהם באי כוח המ...


In [25]:
# נציג 3 דוגמאות
for i in range(3):
    print(f"פסק דין {i+1} (תוצאה מסכמת):\n")
    print(df["תוצאה מסכמת"].iloc[i])
    print("="*80)


פסק דין 1 (תוצאה מסכמת):

טקסט שמצביע על תוצאת בקשה: לנוכח כל האמור לעיל – הבקשה למתן רשות ערעור נדחית || בקשה לרשות ערעור התקבלה: נדחה
פסק דין 2 (תוצאה מסכמת):

טקסט שמצביע על תוצאת בקשה: בקשה לעיכוב ביצוע עונש מאסר || רע"פ בקשה אחרת או דלמטה מיוחד: בקשה אחרת || בקשה לרשות ערעור התקבלה: אחר || הערעור התקבל: אחר
פסק דין 3 (תוצאה מסכמת):

טקסט שמצביע על תוצאת בקשה: אשר על כן, הבקשה לרשות ערעור נדחית בזאת || בקשה לרשות ערעור התקבלה: נדחה


In [26]:
pd.set_option('display.max_colwidth', None)

df[["גוף המסמך חתוך", "תוצאה מסכמת"]].sample(5, random_state=42)


,גוף המסמך חתוך,תוצאה מסכמת
255,"['פסק דין בתיק רע""פ 4386/08 בבית המשפט העליון רע""פ 4386/08 בפני: כבוד השופטת א\' פרוקצ\'יה כבוד השופט א\' רובינשטיין כבוד השופט ס\' ג\'ובראן המבקש: גד כהן נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של בית המשפט המחוזי בירושלים מיום 30.4.08 בע""פ 2074/08 שניתן על-ידי כבוד השופט ע\' חבש – סג""נ בשם המבקש: עו""ד דוד גולן בשם המשיבה: עו""ד מיטל בוכמן-שינדל פסק דין לאור הסכמת הצדדים, המוצאת את ביטויה בסעיף 3 לתגובת המשיבה, ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה בירושלים לדיון מחודש בעניינו. ניתן היום, א\' בחשון התשס""ט (30.10.2008). ש ו פ ט ת ש ו פ ט ש ו פ ט _________________________ העותק כפוף לשינויי עריכה וניסוח. 08043860_H04.doc שצ מרכז מידע, טל\' 02-6593666 ; אתר אינטרנט, www.court.gov.il']","הערות תוצאת ערעור: ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה || בקשה לרשות ערעור התקבלה: חלקי || הערעור התקבל: התקבל"
114,"['החלטה בתיק רע""פ 1682/11 בבית המשפט העליון רע""פ 1682/11 בפני: כבוד השופט ס\' ג\'ובראן המבקש: ואיל מזערה נגד המשיבה: מדינת ישראל בקשה לדחיית מועד תחילת ריצוי עונש המאסר בשם המבקש: עו""ד מאליק מנצור החלטה הבקשה נדחית. ניתנה היום, כ""ה באדר ב התשע""א (31.3.2011). ש ו פ ט _________________________ העותק כפוף לשינויי עריכה וניסוח. 11016820_H02.doc שצ מרכז מידע, טל\' 077-2703333 ; אתר אינטרנט, www.court.gov.il']","רע""פ בקשה אחרת או דלמטה מיוחד: בקשה אחרת || הערות: בקשה לדחיית מועד תחילת ריצוי עונש המאסר || בקשה לרשות ערעור התקבלה: נדחה"
314,"['החלטה בתיק בש""פ 1092/17 בבית המשפט העליון רע""פ 1092/17 לפני: כבוד השופטת ד\' ברק-ארז המערערים: 1. כפיר דהן 2. שירה דהן נגד המשיבה: הוועדה המקומית לתכנון ובניה מטה יהודה בקשת רשות ערעור על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 בע""פ 23940-12-16 שניתן על-ידי כבוד השופט ח\' מ\' לומפ בשם המערערים: בעצמם בשם המשיבה: עו""ד רעות אהרון, עו""ד מאיר בן דוד פסק דין 1. הבקשה שבפני נקלטה במזכירות בית המשפט, בטעות, תחת הסיווג בש""פ. הלכה למעשה, מדובר בבקשת רשות לערער על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 (ע""פ 23940-12-16, השופטת ח\' מ\' לומפ). סיווגה הראוי אפוא הוא רע""פ, וכך אני מורה. 2. מקורו של ההליך דנן הוא בבקשה למתן אורכה לביצועו של צו ההריסה שהוצא ביחס למבנה היביל שבו מתגוררים המבקשים במסגרת הליך פלילי לפי חוק התכנון והבניה, התשכ""ה-1965. הבקשה נדחתה על ידי בית המשפט לעניינים מקומיים בבית שמש ועל כן הגישו המבקשים ערעור לבית המשפט המחוזי, אשר קבע כי תינתן למבקשים אורכה לפנות את המבנה עד ליום 1.4.2017 ולהרסו עד ליום 2.5.2017. לאחר מכן המבקשים הגישו בקשת אורכה נוספת לבית המשפט המחוזי, וזו נדחתה. על כך נסבה הבקשה שבפני. 3. בתשובה שהוגשה מטעם המשיבה צוין כי המקרה העומד ביסוד ההליך כרוך בנסיבות אישיות ומשפחתיות חריגות וקשות ועל כן היא מבקשת להותיר את ההכרעה בו לשיקול דעתו של בית המשפט. בהתחשב בכך ולאחר שבחנתי את מכלול הנסיבות העניין, אני מורה על מתן אורכה נוספת למבקשים לפנות את המבנה עד ליום 1.7.2017 ולהרסו עד ליום 2.8.2017 (בתנאים שנקבעו בפסק דינו של בית המשפט המחוזי). 4. הבקשה מתקבלת אפוא כמפורט בפסקה 3. על המבקשים להבין כי ההיענות לבקשתם התאפשרה לנוכח העמדה שגילתה המשיבה, וכי אל להם לצפות לכל הקלות נוספות, אלא להיערך לביצוע הצו השיפוטי, משתמו ההליכים כולם. ניתן היום, \u200fי""ט בשבט התשע""ז (\u200f15.2.2017). ש ו פ ט ת _________________________ העותק כפוף לשינויי עריכה וניסוח. 17010920_A02.doc עכ מרכז מידע, טל\' 077-2703333 ; אתר אינטרנט, www.court.gov.il']",הערות: ביהמ׳׳ש נתן אורכה אך לא ביטל את צו ההריסה || הערות תוצאת ערעור: הבקשה מתקבלת אפוא כמפורט בפסקה 3 || בקשה לרשות ערעור התקבלה: התקבל || הערעור התקבל: חלקי
268,"['פסק דין בתיק רע""פ 1793/09 בבית המשפט העליון רע""פ 1793/09 בפני: כבוד השופט א\' א\' לוי כבוד השופטת ע\' ארבל כבוד השופט י\' דנציגר המבקשים: 1. פאבל מינסטר 2. זאב שוורץ נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של ביתה משפט המחוזי בתל-אביב, בע""פ 70149/08, מיום 28.1.09, שניתן על ידי כבוד ס\' הנשיא השופטת ד\' ברלינר, ס. הנשיא השופט ז\' המר, ס. הנשיא השופטת ב\' אופיר-תום בשם המבקש 1: עו""

In [27]:

def clean_text(text):
    if pd.isna(text):  # אם הערך ריק, נחזיר אותו כמו שהוא
        return text

    text = re.sub(r"העותק כפוף לשינויי עריכה וניסוח.*?$", "", text, flags=re.MULTILINE)
    text = re.sub(r"מרכז מידע, טל' \d{2,3}-\d{6,7}.*?$", "", text, flags=re.MULTILINE)
    text = re.sub(r"אתר אינטרנט, .*?$", "", text, flags=re.MULTILINE)

    return text.strip()  # מסירים רווחים מיותרים

# החלת הניקוי על גוף המסמך החתוך
df["גוף המסמך חתוך"] = df["גוף המסמך חתוך"].apply(clean_text)

def clean_result(text):
    if pd.isna(text):
        return text

    # שמירה רק על התוצאה של הערעור
    match = re.search(r"(?i)(בקשה לרשות ערעור התקבלה:.*?)(?:\|\||$)", text)
    return match.group(1) if match else text.strip()

df["תוצאה מסכמת"] = df["תוצאה מסכמת"].apply(clean_result)



<ipython-input-27-a91d50effff6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["גוף המסמך חתוך"] = df["גוף המסמך חתוך"].apply(clean_text)
<ipython-input-27-a91d50effff6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["תוצאה מסכמת"] = df["תוצאה מסכמת"].apply(clean_result)


In [28]:
df[["גוף המסמך חתוך", "תוצאה מסכמת"]].sample(5, random_state=42)


,גוף המסמך חתוך,תוצאה מסכמת
255,"['פסק דין בתיק רע""פ 4386/08 בבית המשפט העליון רע""פ 4386/08 בפני: כבוד השופטת א\' פרוקצ\'יה כבוד השופט א\' רובינשטיין כבוד השופט ס\' ג\'ובראן המבקש: גד כהן נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של בית המשפט המחוזי בירושלים מיום 30.4.08 בע""פ 2074/08 שניתן על-ידי כבוד השופט ע\' חבש – סג""נ בשם המבקש: עו""ד דוד גולן בשם המשיבה: עו""ד מיטל בוכמן-שינדל פסק דין לאור הסכמת הצדדים, המוצאת את ביטויה בסעיף 3 לתגובת המשיבה, ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה בירושלים לדיון מחודש בעניינו. ניתן היום, א\' בחשון התשס""ט (30.10.2008). ש ו פ ט ת ש ו פ ט ש ו פ ט _________________________",בקשה לרשות ערעור התקבלה: חלקי
114,"['החלטה בתיק רע""פ 1682/11 בבית המשפט העליון רע""פ 1682/11 בפני: כבוד השופט ס\' ג\'ובראן המבקש: ואיל מזערה נגד המשיבה: מדינת ישראל בקשה לדחיית מועד תחילת ריצוי עונש המאסר בשם המבקש: עו""ד מאליק מנצור החלטה הבקשה נדחית. ניתנה היום, כ""ה באדר ב התשע""א (31.3.2011). ש ו פ ט _________________________",בקשה לרשות ערעור התקבלה: נדחה
314,"['החלטה בתיק בש""פ 1092/17 בבית המשפט העליון רע""פ 1092/17 לפני: כבוד השופטת ד\' ברק-ארז המערערים: 1. כפיר דהן 2. שירה דהן נגד המשיבה: הוועדה המקומית לתכנון ובניה מטה יהודה בקשת רשות ערעור על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 בע""פ 23940-12-16 שניתן על-ידי כבוד השופט ח\' מ\' לומפ בשם המערערים: בעצמם בשם המשיבה: עו""ד רעות אהרון, עו""ד מאיר בן דוד פסק דין 1. הבקשה שבפני נקלטה במזכירות בית המשפט, בטעות, תחת הסיווג בש""פ. הלכה למעשה, מדובר בבקשת רשות לערער על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 (ע""פ 23940-12-16, השופטת ח\' מ\' לומפ). סיווגה הראוי אפוא הוא רע""פ, וכך אני מורה. 2. מקורו של ההליך דנן הוא בבקשה למתן אורכה לביצועו של צו ההריסה שהוצא ביחס למבנה היביל שבו מתגוררים המבקשים במסגרת הליך פלילי לפי חוק התכנון והבניה, התשכ""ה-1965. הבקשה נדחתה על ידי בית המשפט לעניינים מקומיים בבית שמש ועל כן הגישו המבקשים ערעור לבית המשפט המחוזי, אשר קבע כי תינתן למבקשים אורכה לפנות את המבנה עד ליום 1.4.2017 ולהרסו עד ליום 2.5.2017. לאחר מכן המבקשים הגישו בקשת אורכה נוספת לבית המשפט המחוזי, וזו נדחתה. על כך נסבה הבקשה שבפני. 3. בתשובה שהוגשה מטעם המשיבה צוין כי המקרה העומד ביסוד ההליך כרוך בנסיבות אישיות ומשפחתיות חריגות וקשות ועל כן היא מבקשת להותיר את ההכרעה בו לשיקול דעתו של בית המשפט. בהתחשב בכך ולאחר שבחנתי את מכלול הנסיבות העניין, אני מורה על מתן אורכה נוספת למבקשים לפנות את המבנה עד ליום 1.7.2017 ולהרסו עד ליום 2.8.2017 (בתנאים שנקבעו בפסק דינו של בית המשפט המחוזי). 4. הבקשה מתקבלת אפוא כמפורט בפסקה 3. על המבקשים להבין כי ההיענות לבקשתם התאפשרה לנוכח העמדה שגילתה המשיבה, וכי אל להם לצפות לכל הקלות נוספות, אלא להיערך לביצוע הצו השיפוטי, משתמו ההליכים כולם. ניתן היום, \u200fי""ט בשבט התשע""ז (\u200f15.2.2017). ש ו פ ט ת _________________________",בקשה לרשות ערעור התקבלה: התקבל
268,"['פסק דין בתיק רע""פ 1793/09 בבית המשפט העליון רע""פ 1793/09 בפני: כבוד השופט א\' א\' לוי כבוד השופטת ע\' ארבל כבוד השופט י\' דנציגר המבקשים: 1. פאבל מינסטר 2. זאב שוורץ נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של ביתה משפט המחוזי בתל-אביב, בע""פ 70149/08, מיום 28.1.09, שניתן על ידי כבוד ס\' הנשיא השופטת ד\' ברלינר, ס. הנשיא השופט ז\' המר, ס. הנשיא השופטת ב\' אופיר-תום בשם המבקש 1: עו""ד אמיר ציון בשם המבקש 2: עו""ד רובינשטיין מנחם פסק דין נוכח האמור בתגובת המשיבה, ניתנת בזאת רשות ערעור והבקשה תידון כבערעור גופו. עניינם של המבקשים יובא בשנית בפני בית המשפט המחוזי, המתבקש להבהיר אם תוצאת פסק-דינו היא זיכוי מוחלט של המבקשים או זיכוי מחמת הספק. ניתן היום, ט""ו באדר התשס""ט (11.03.09). ש ו פ ט ש ו פ ט ת ש ו פ ט _________________________",בקשה לרשות ערעור התקבלה: התקבל
167,"['בבית המשפט העליון בירושלים רע""פ 9608/01 בפני: כבוד השופט א\' ריבלין המבקשים: 1. גשמי ברכה ה.ש.י.ג.(1995) 2. גשמי ברכה בע""מ 3. ה.ש.י.ג.(1995)יזום פרוייקטים 4. שנהב זאב נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של בית המשפט המחוזי בתל-אביב-יפו מיום 23.10.2001 בתיק עפ""א 80190/01 שניתן על ידי כבוד השופטים: ד\' ברלינר, ז\' המר, י\' שטופמן בשם המבקשים: עו""

In [29]:
df.to_excel("/content/dataframe_cleaned.xlsx", index=False)

In [30]:

with open("/content/api_key.txt", "r") as file:
    api_key = file.read().strip()

# יצירת לקוח OpenAI
client = openai.OpenAI(api_key=api_key)


response = client.chat.completions.create(
    model="gpt-4o",  # ניתן להחליף ל-"gpt-4o-mini" לחיסכון במשאבים
    messages=[
        {"role": "system", "content": "אתה עוזר משפטי המתמחה בניתוח פסקי דין בעברית."},
        {"role": "user", "content": "פסק הדין: פלוני נגד מדינת ישראל, הנאשם טען כי ... כיצד פסק הדין מסתיים?"}
    ],
    max_tokens=200
)


print(response.choices[0].message.content)

אין ביכולתי לספק את תוכן הסיום של פסק הדין הספציפי "פלוני נגד מדינת ישראל" ללא פרטים נוספים על המקרה, שכן אין לי גישה למאגרי מידע בזמן אמת. פסקי דין מתפרסמים לעיתים במאגרים משפטיים או באתרי בתי המשפט, ומומלץ לבדוק שם ישירות. אם יש לך שאלות כלליות או פרטים נוספים על המקרה, אוכל לנסות לסייע עם הניתוח.


In [37]:

train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# פונקציה להמרת הדאטה לפורמט JSONL של OpenAI
def convert_to_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for _, row in data.iterrows():
            json.dump({
                "messages": [
                    {"role": "system", "content": "אתה עוזר משפטי לניתוח פסקי דין מסוג רע\"פ"},
                    {"role": "user", "content": row["גוף המסמך חתוך"]},
                    {"role": "assistant", "content": row["תוצאה מסכמת"]}
                ]
            }, f, ensure_ascii=False)
            f.write("\n")

# יצירת קבצים
convert_to_jsonl(train_data, "train.jsonl")
convert_to_jsonl(valid_data, "valid.jsonl")
convert_to_jsonl(test_data, "test.jsonl")

print(" הנתונים מוכנים ל-Fine-Tuning!")


 הנתונים מוכנים ל-Fine-Tuning!


In [48]:
train_file = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)

print("Train File ID:", train_file.id)


Train File ID: file-53Ns892y8pFTfiYRrBJ8ki


In [49]:
files = client.files.list()
print(files)


<frozen abc>:123: RuntimeWarning: coroutine 'upload_file' was never awaited


SyncCursorPage[FileObject](data=[FileObject(id='file-53Ns892y8pFTfiYRrBJ8ki', bytes=1108947, created_at=1740489359, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-4uQENmlKIEQklPGwd7nccNbo', bytes=351964, created_at=1700573627, filename='COCKTAIL MENU EXAMLPLE.PNG', object='file', purpose='assistants', status='processed', status_details=None, expires_at=None)], has_more=False, object='list', first_id='file-53Ns892y8pFTfiYRrBJ8ki', last_id='file-4uQENmlKIEQklPGwd7nccNbo')


In [51]:
fine_tune = client.fine_tuning.jobs.create(
    training_file="file-53Ns892y8pFTfiYRrBJ8ki",  # ID של קובץ האימון שהועלה
    model="gpt-3.5-turbo"
)

print("Fine-Tuning Job ID:", fine_tune.id)


Fine-Tuning Job ID: ftjob-6I2ymZHMQd7DWCCORN2eha9n


In [53]:
fine_tune_status = client.fine_tuning.jobs.retrieve(fine_tune.id)
print(fine_tune_status)


FineTuningJob(id='ftjob-6I2ymZHMQd7DWCCORN2eha9n', created_at=1740489420, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XsklJyn5A831tAwBRgKN0tGP', result_files=[], seed=2036021448, status='running', trained_tokens=None, training_file='file-53Ns892y8pFTfiYRrBJ8ki', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None)


In [56]:
fine_tune_job = client.fine_tuning.jobs.retrieve("ftjob-6I2ymZHMQd7DWCCORN2eha9n")
print(fine_tune_job.status)


running


In [ ]:

job_id = "ftjob-6I2ymZHMQd7DWCCORN2eha9n"
while True:
    fine_tune_job = client.fine_tuning.jobs.retrieve(job_id)
    print(f"Status: {fine_tune_job.status}")

    if fine_tune_job.status in ["succeeded", "failed", "cancelled"]:
        print(f"Fine-Tuned Model ID: {fine_tune_job.fine_tuned_model}")
        break

    time.sleep(60*5)

Status: running


In [ ]:
print(fine_tune_job.trained_tokens)
print(fine_tune_job.fine_tuned_model)

In [ ]:
result_files = client.files.list()
for file in result_files.data:
    if "fine-tuning" in file.purpose:
        print(file.filename, file.id)


In [ ]:
response = client.chat.completions.create(
    model=f"ft:gpt-3.5-turbo-0125:{fine_tune_job.fine_tuned_model}"
    messages=[
        {"role": "system", "content": "אתה עוזר משפטי לניתוח פסקי דין מסוג רע\"פ"},
        {"role": "user", "content": test_data.iloc[0]["גוף המסמך חתוך"]}
    ]
)

print(response.choices[0].message.content)
